In [42]:
def print_product_properties(product):
    print("Product Name:", product.get('p_name', 'N/A'))
    print("Type:", product.get('p_type', 'N/A'))
    print("Brand:", product.get('p_brand', 'N/A'))
    print("is anibiotic:", product.get('is_antibiotics', 'N/A'))
    print("Description:", product.get('p_description', 'N/A'))
    print("Meta Description:", product.get('p_meta_description', 'N/A'))
    print("Keywords:", product.get('p_product_keywords', 'N/A'))
    print("Manufacturer:", product.get('p_manufacturer', 'N/A'))
    print("Product Cold Chain:", product.get('p_cold', 'N/A'))
    print("Tags:", product.get('p_product_tags_title', 'N/A'))
    
    if 'pv' in product and len(product['pv']) > 0:
        pv_item = product['pv'][0]
        print("Base Unit Label:", pv_item.get("pu_base_unit_label", 'N/A'))
        print("Average Rating:", pv_item.get("pv_average_rating", 'N/A'))
        print("MRP:", pv_item.get("pv_mrp", 'N/A'))
    
    print("\n")

In [ ]:
# Now save the data in a CSV file convert array into a single string as well as the htmls

import csv
import requests
import json

api_url = "https://api.arogga.com/general/v3/search/?_page=3&_perPage=100"

# Make the request
response = requests.get(api_url)

# Path to the CSV file
csv_file_path = 'k:/Other Drive Copy/Code/GITHUB_REPOS/poridhi-ai-engineering-hackathon/scrapper/arogga/genera-v3-search-category.csv'

# Open the CSV file
data = response.json()
with open(csv_file_path, 'a', newline='' , encoding='utf-8') as file:

    # Create the CSV writer
    writer = csv.writer(file)

    # Write the header
    writer.writerow([
        'Product Name',
        'Type',
        'Brand',
        'Is Anti-Biotic',
        'Description',
        'Meta Description',
        'Keywords',
        'Manufacturer',
        'Product Cold Chain',
        'Tags',
        'Base Unit Label',
        'Average Rating',
        'MRP'
    ])

    # Write the data
    for product in data['data']:
        if 'pv' in product and len(product['pv']) > 0:
            pv_item = product['pv'][0]
            writer.writerow([
                product.get('p_name', 'N/A'),
                product.get('p_type', 'N/A'),
                product.get('p_brand', 'N/A'),
                product.get('is_antibiotics', 'N/A'),
                product.get('p_description', 'N/A'),
                product.get('p_meta_description', 'N/A'),
                product.get('p_product_keywords', 'N/A'),
                product.get('p_manufacturer', 'N/A'),
                product.get('p_cold', 'N/A'),
                product.get('p_product_tags_title', 'N/A'),
                pv_item.get("pu_base_unit_label", 'N/A'),
                pv_item.get("pv_average_rating", 'N/A'),
                pv_item.get("pv_mrp", 'N/A')
            ])
        else:
            writer.writerow([
                product.get('p_name', 'N/A'),
                product.get('p_type', 'N/A'),
                product.get('p_brand', 'N/A'),
                product.get('is_antibiotics', 'N/A'),
                product.get('p_description', 'N/A'),
                product.get('p_meta_description', 'N/A'),
                product.get('p_product_keywords', 'N/A'),
                product.get('p_manufacturer', 'N/A'),
                product.get('p_cold', 'N/A'),
                product.get('p_product_tags_title', 'N/A'),
                'N/A',
                'N/A',
                'N/A'
            ])

# Now automate the process of

{
    "total": 1000,
    "status": "success",
    "data": [
        {
            "pv": [
                {
                    "pv_id": 7730,
                    "pu_base_unit_label": "15ml drop",
                    "pu_b2c_sales_unit_label": "15ml drop",
                    "pu_b2b_sales_unit_label": "15ml drop",
                    "pv_attribute": "",
                    "pv_sku": "",
                    "pv_total_rating": 0,
                    "pv_average_rating": 0,
                    "pv_p_id": 7730,
                    "pv_medicine_id": 7730,
                    "pv_mrp": 40,
                    "pv_b2c_discounted_price": 36.36,
                    "pv_b2c_min_qty": 1,
                    "pv_b2c_max_qty": 200,
                    "pv_b2b_discounted_price": 35.552,
                    "pv_b2b_min_qty": 1,
                    "pv_b2b_max_qty": 400,
                    "pv_allow_purchase": 1,
                    "pv_allow_sales": 1,
                    "pv_is_base": 1,
        

In [ ]:
import csv
import requests
import json
import time

# Path to the CSV file
csv_file_path = 'k:/Other Drive Copy/Code/GITHUB_REPOS/poridhi-ai-engineering-hackathon/scrapper/arogga/genera-v3-search-category.csv'

# Open the CSV file

with open(csv_file_path, 'w', newline='' , encoding='utf-8') as file:
    
        # Create the CSV writer
        writer = csv.writer(file)
    
        # Write the header
        writer.writerow([
            'Product Name',
            'Type',
            'Brand',
            'Is Anti-Biotic',
            'Description',
            'Meta Description',
            'Keywords',
            'Manufacturer',
            'Product Cold Chain',
            'Tags',
            'Base Unit Label',
            'Average Rating',
            'MRP'
        ])
    
        for page in range(1, 400):
            api_url = f"https://api.arogga.com/general/v3/search/?_page={page}&_perPage=100"
            # Make the request
            response = requests.get(api_url)
    
            # Write the data
            data = response.json()
            for product in data['data']:
                if 'pv' in product and len(product['pv']) > 0:
                    pv_item = product['pv'][0]
                    writer.writerow([
                        product.get('p_name', 'N/A'),
                        product.get('p_type', 'N/A'),
                        product.get('p_brand', 'N/A'),
                        product.get('is_antibiotics', 'N/A'),
                        product.get('p_description', 'N/A'),
                        product.get('p_meta_description', 'N/A'),
                        product.get('p_product_keywords', 'N/A'),
                        product.get('p_manufacturer', 'N/A'),
                        product.get('p_cold', 'N/A'),
                        product.get('p_product_tags_title', 'N/A'),
                        pv_item.get("pu_base_unit_label", 'N/A'),
                        pv_item.get("pv_average_rating", 'N/A'),
                        pv_item.get("pv_mrp", 'N/A')
                    ])
                else:
                    writer.writerow([
                        product.get('p_name', 'N/A'),
                        product.get('p_type', 'N/A'),
                        product.get('p_brand', 'N/A'),
                        product.get('is_antibiotics', 'N/A'),
                        product.get('p_description', 'N/A'),
                        product.get('p_meta_description', 'N/A'),
                        product.get('p_product_keywords', 'N/A'),
                        product.get('p_manufacturer', 'N/A'),
                        product.get('p_cold', 'N/A'),
                        product.get('p_product_tags_title', 'N/A'),
                        'N/A',
                        'N/A',
                        'N/A'
                    ])
            time.sleep(7)

